In [176]:

import sys

from google.api_core.client_options import ClientOptions
from google.cloud import automl_v1
from google.cloud.automl_v1.proto import service_pb2

import re
import pandas as pd

In [7]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALs'] = 'C:/Users/chpar10/OneDrive - IKEA/DA/service_key.json'

In [5]:
def inline_text_payload(file_path):
    with open(file_path, 'rb') as ff:
        content = ff.read()
    return {'text_snippet': {'content': content, 'mime_type': 'text/plain'} }

def pdf_payload(file_path):
    return {'document': {'input_config': {'gcs_source': {'input_uris': [file_path] } } } }

def get_prediction(file_path, model_name):
    options = ClientOptions(api_endpoint='automl.googleapis.com' )
    prediction_client = automl_v1.PredictionServiceClient(client_options=options)

    payload = inline_text_payload(file_path)
  # Uncomment the following line (and comment the above line) if want to predict on PDFs.
  # payload = pdf_payload(file_path)

    params = {}
    request = prediction_client.predict(model_name, payload, params)
    return request  # waits until request is returned

if __name__ == '__main__':
    file_path = sys.argv[0]
    model_name = sys.argv[1]

In [104]:
flist = os.listdir()
flist = [os.getcwd()+'/'+i for i in flist if recomp.search(i)]

In [120]:
os.listdir('c:/')

['$Recycle.Bin',
 '0ABD25F26AB8',
 'db',
 'Documents and Settings',
 'drivers10',
 'Drv',
 'hiberfil.sys',
 'IKEALogs',
 'inetpub',
 'Intel',
 'javalog',
 'OneDriveTemp',
 'orarun',
 'pagefile.sys',
 'PerfLogs',
 'Program Files',
 'Program Files (x86)',
 'ProgramData',
 'Recovery',
 'swapfile.sys',
 'System Volume Information',
 'Users',
 'Windows']

In [94]:
recomp = re.compile('[.]txt')
recomp.search('.ipynb_checkpoints.txt')

<re.Match object; span=(18, 22), match='.txt'>

In [188]:
def get_result(directory=None):
    flist = os.listdir(directory)
    flist = [directory+'/'+i for i in flist if recomp.search(i)]
    tmp_dict = {'comment':[], 'cat1':[], 'cat2':[]}
    
    for file in flist:
        tmp = get_prediction(file, 'projects/721541100077/locations/us-central1/models/TCN5423015648563298304')
        
        with open(file, 'r', encoding = 'utf-8') as f:
            tmp_dict['comment'].append(re.sub('\ufeff',"",f.read()))
        cat = re.findall("[A-Z][^A-Z]*",tmp.payload[0].display_name)
        tmp_dict['cat1'].append(cat[0])
        tmp_dict['cat2'].append(''.join(cat[1:]))
                               
    return tmp_dict

In [170]:
outcome = get_result("C:/Users/chpar10/OneDrive - IKEA/DA/Text Analysis on freetext/sample")

In [172]:
outcome

{'comment': ['배송 지연 때문에 맘이 상했네요',
  'Malm 침대 조립 중 드는 생각은. 합판의 강도를 높일 필요성이 있습니다. 나사를 4개를 장착하면 그중에 하나는 헛도는 느낌이 좀 나네요.\n'],
 'cat1': ['Service', 'Product'],
 'cat2': ['Quality', 'Quality']}

In [174]:
import pandas as pd

In [175]:
pd.DataFrame(outcome)

,comment,cat1,cat2
0,배송 지연 때문에 맘이 상했네요,Service,Quality
1,Malm 침대 조립 중 드는 생각은. 합판의 강도를 높일 필요성이 있습니다. 나사를...,Product,Quality
